# Sparse Non-Negative PARAFAC

This notebook is based on [sparse_demo.ipynb](sparse_demo.ipynb#parafac). 

As before, we start with a random sparse tensor, constructed so that it has a tensor factorization of rank 5.

Because non-negative PARAFAC can take longer to converge than non-masked PARAFAC and also produce dense factors, we will use a smaller tensor than in the other notebook.

In [3]:
shape = (1000, 1001, 1002)
rank = 5

import sparse
starting_weights = sparse.ones(rank)
starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors

[<COO: shape=(1000, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1001, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1002, 5), dtype=float64, nnz=50, fill_value=0.0>]

In [4]:
from tensorly.contrib.sparse.kruskal_tensor import kruskal_to_tensor
tensor = kruskal_to_tensor((starting_weights, starting_factors))
tensor

Format,coo
Data Type,float64
Shape,"(1000, 1001, 1002)"
nnz,5091
Density,5.075762560792501e-06
Read-only,True
Size,159.1K
Storage ratio,0.0


In [5]:
tensor.nbytes / 1e9                # Actual memory usage in GB

0.000162912

In [6]:
import numpy as np
np.prod(tensor.shape) * 8 / 1e9    # Memory usage if array was dense, in GB

8.024016

Now we factor the tensor. Note that at this time, you have to use the `non_negative_parafac` function from the sparse backend when using a sparse mask to avoid memory blowups.

In [7]:
import time
%load_ext memory_profiler
from tensorly.contrib.sparse.decomposition import non_negative_parafac

In [14]:
%%memit
start_time = time.time()
sparse_kruskal = non_negative_parafac(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

reconstruction error=0.9160916858980143
iteration 1, reconstraction error: 0.7563893341901149, decrease = 0.15970235170789937
iteration 2, reconstraction error: 0.7238710798828315, decrease = 0.03251825430728339
iteration 3, reconstraction error: 0.638136189700943, decrease = 0.08573489018188851
iteration 4, reconstraction error: 0.6237710805095488, decrease = 0.01436510919139422
iteration 5, reconstraction error: 0.6230289529902983, decrease = 0.0007421275192505128
iteration 6, reconstraction error: 0.6225200750934725, decrease = 0.0005088778968257435
iteration 7, reconstraction error: 0.6221525910453113, decrease = 0.00036748404816122626
iteration 8, reconstraction error: 0.6218784625314465, decrease = 0.00027412851386476866
iteration 9, reconstraction error: 0.6216689973699929, decrease = 0.0002094651614535925
iteration 10, reconstraction error: 0.6215059333127146, decrease = 0.00016306405727828377
iteration 11, reconstraction error: 0.6213770599793786, decrease = 0.0001288733333359

Let's look at one of the values that was masked out.

In [15]:
tensor.coords.T[0]

array([ 18, 116,  45])

In [16]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.5859621618332147

See the [sparse_demo.ipynb](sparse_demo.ipynb) for how to calculate individual values from the factors. Note that we do not compare the entire tensor because it would be dense, and memory usage would be prohibitive.

In [17]:
weights, factors = sparse_kruskal
computed_val = np.sum(np.prod(sparse.stack([factors[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
computed_val

2.2044092111848095e-32

In [18]:
np.abs(orig_val - computed_val)

0.5859621618332147

In [19]:
[f.density for f in factors]

[1.0, 1.0, 1.0]